# Funeral home data inspection, normalization, deduplication, and load

This notebook:

1. Loads two JSON sources:
   - `funeral_home_listings.json`
   - `funeral_home_locations.json`
2. Loads the existing legacy SQLite database:
   - `funeral_homes_legacy.db`
3. Normalizes all sources into a single internal schema.
4. Deduplicates records (tolerant to slight name/address differences).
5. Writes the unified, deduped dataset back to a database (SQLite by default; Postgres optional).

---

## Files
- `funeral_home_listings.json`
- `funeral_home_locations.json`
- `funeral_homes_legacy.db`


In [1]:
# --- Setup ---
import json
import re
import math
import sqlite3
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

LISTINGS_PATH = Path("data/funeral_home_listings.json")
LOCATIONS_PATH = Path("data/funeral_home_locations.json")
LEGACY_DB_PATH = Path("data/funeral_homes_legacy.db")

assert LISTINGS_PATH.exists(), f"Missing {LISTINGS_PATH}"
assert LOCATIONS_PATH.exists(), f"Missing {LOCATIONS_PATH}"
assert LEGACY_DB_PATH.exists(), f"Missing {LEGACY_DB_PATH}"

print("OK: files found")


OK: files found


In [2]:
# --- Load JSON sources ---
with LISTINGS_PATH.open("r", encoding="utf-8") as f:
    listings_raw = json.load(f)

with LOCATIONS_PATH.open("r", encoding="utf-8") as f:
    locations_raw = json.load(f)

print("Listings records:", len(listings_raw))
print("Locations records:", len(locations_raw))

# quick look
pd.DataFrame(listings_raw).head(3)


Listings records: 3019
Locations records: 8265


,name,learnMoreLink,website,address,phone,contactPerson,contactLink,memberSince,affiliations
0,El Camino Memorial Park,/canamembers/Cemetery/El-Camino-Memorial-Park-...,NaN,"{'street': '5600 Carroll Canyon Rd', 'locality...",(718) 633-0400,Douglas Trobaugh,/directory/directoryemailform.aspx?mcid=27160A...,2015,[]
1,Cementerio Parque De Luz,/canamembers/Cemetery/Cementerio-Parque-De-Luz...,http://www.cementerioparquedeluz.com,"{'street': 'Carr 1, KM 29.2', 'locality': 'Cag...",(787) 745-3200,Jennifer Ramos-Crespo,/directory/directoryemailform.aspx?mcid=21100B...,2015,[]
2,Coral Ridge Funeral Home & Cemetery,/canamembers/Cemetery/Coral-Ridge-Funeral-Home...,NaN,"{'street': '1630 SW Pine Island Rd', 'locality...",(423) 698-2541,Charles Warren,/directory/directoryemailform.aspx?mcid=221X0M...,2015,[]


In [3]:
pd.DataFrame(locations_raw).head(3)


,id,title,latitude,longitude,description,phone,websiteUrl,address,directionsUrl
0,a93d2885-5b48-ef11-a317-7c1e5216a74f,1st Choice Mortuary,32.23935,-90.23272,,601-346-5738,,"3989 Terry Rd , Jackson, 39212-5266, US",
1,86e42e0e-f167-f011-bec2-000d3a180c28,302Funerals / Williams Funeral Home and Buria...,39.74206,-75.58487,,,https://www.302funerals.com/,"2100 Rodman Rd , Wilmington, 19805-4133, US",
2,27e9233e-93b8-db11-b90e-0050568763a2,A A Mariani & Son Funeral Home,41.85052,-71.42264,,401-861-5432,https://www.marianiandson.com/,"200 Hawkins St , Providence, 02904-2216, US",


## Inspect and profile source schemas

We'll inspect the columns present, null rates, and a few example values to understand what we need to normalize.


In [4]:
df_listings_raw = pd.json_normalize(listings_raw)
df_locations_raw = pd.json_normalize(locations_raw)

def profile(df, name, n=10):
    print(f"\n=== {name} ===")
    print("shape:", df.shape)
    display(df.head(3))
    print("\nColumns:", list(df.columns))
    null_rate = (df.isna().mean().sort_values(ascending=False) * 100).round(1)
    display(null_rate.head(25).to_frame("null_%"))
    display(df.sample(min(n, len(df)), random_state=0))

profile(df_listings_raw, "listings_raw")



=== listings_raw ===
shape: (3019, 13)


,name,learnMoreLink,website,phone,contactPerson,contactLink,memberSince,affiliations,address.street,address.locality,address.region,address.postalCode,address.country
0,El Camino Memorial Park,/canamembers/Cemetery/El-Camino-Memorial-Park-...,NaN,(718) 633-0400,Douglas Trobaugh,/directory/directoryemailform.aspx?mcid=27160A...,2015,[],5600 Carroll Canyon Rd,San Diego,CA,92121,None
1,Cementerio Parque De Luz,/canamembers/Cemetery/Cementerio-Parque-De-Luz...,http://www.cementerioparquedeluz.com,(787) 745-3200,Jennifer Ramos-Crespo,/directory/directoryemailform.aspx?mcid=21100B...,2015,[],"Carr 1, KM 29.2",Caguas,PR,00726 1569,None
2,Coral Ridge Funeral Home & Cemetery,/canamembers/Cemetery/Coral-Ridge-Funeral-Home...,NaN,(423) 698-2541,Charles Warren,/directory/directoryemailform.aspx?mcid=221X0M...,2015,[],1630 SW Pine Island Rd,Cape Coral,FL,33991,None



Columns: ['name', 'learnMoreLink', 'website', 'phone', 'contactPerson', 'contactLink', 'memberSince', 'affiliations', 'address.street', 'address.locality', 'address.region', 'address.postalCode', 'address.country']


,null_%
address.country,100.0
website,29.2
contactLink,7.6
phone,2.5
contactPerson,1.2
learnMoreLink,0.0
name,0.0
affiliations,0.0
memberSince,0.0
address.street,0.0


,name,learnMoreLink,website,phone,contactPerson,contactLink,memberSince,affiliations,address.street,address.locality,address.region,address.postalCode,address.country
663,Ballweg & Lunsford Funeral Homes - Syracuse,/canamembers/Funeral-Home/Ballweg-Lunsford-Fun...,https://www.ballweg-lunsford.com/,(315) 492-3504,Tzewei Sung,/directory/directoryemailform.aspx?mcid=27201M...,2016,[Funeral Home],4612 S. Salina Street,Syracuse,NY,13205,None
156,T. J. McGowan Sons Funeral Home - Garnerville,/canamembers/Funeral-Home/T-J-McGowan-Sons-Fun...,https://www.tjmcgowansonsfh.com/,(845) 429-6665,Leanna Wegielski,/directory/directoryemailform.aspx?mcid=282H10...,2015,[Funeral Home],71 North Central Highway,Garnerville,NY,10923,None
2620,"Tharp Funeral Home & Crematory, Inc.",/canamembers/Funeral-Home/Tharp-Funeral-Home-C...,http://www.tharpfuneralhome.com,(434) 237-9424,Chris Tharp,/directory/directoryemailform.aspx?mcid=875H8W...,1997,"[Crematory, Funeral Home]",220 Breezewood Drive,Lynchburg,VA,24502 4504,None
2422,Greenwood Serenity Memorial Gardens,/canamembers/Cemetery/Greenwood-Serenity-Memor...,NaN,(865) 970-2955,Brandy Blakely,/directory/directoryemailform.aspx?mcid=2U1G0X...,2015,[],909 Lincoln Road,Montgomery,AL,36109,None
1158,Riverside Memorial Chapels of New Jersey,/canamembers/Funeral-Home/Riverside-Memorial-C...,http://www.riversidememorialhackensack.com,(201) 489-2728,Charles Costello,/directory/directoryemailform.aspx?mcid=2Z120C...,2015,"[Crematory, Funeral Home, Mortuary]",402 Park Street,Hackensack,NJ,07601,None
1129,Cremation Society of Pennsylvania - Pittsburgh,/canamembers/Cremation-Society/Cremation-Socie...,NaN,(878) 332-5527,Jon O'data,/directory/directoryemailform.aspx?mcid=2L1403...,2018,[Cremation Society],"11269 Perry Highway, Suite 222",Pittsburgh,PA,15090,None
2063,Moshassuck Cemetery Corporation,/canamembers/Crematory/Moshassuck-Cemetery-Cor...,NaN,(401) 723-1087,George H. Boardman,/directory/directoryemailform.aspx?mcid=301I20...,1984,[Crematory],978 Lonsdale Ave,Central Falls,RI,02863 1719,None
1276,I. J. Morris Funeral Directors,/canamembers/Funeral-Home/I-J-Morris-Funeral-D...,http://www.ijmorriswestpalmbeach.com,(561) 683-8676,Jonathan Weis,/directory/directoryemailform.aspx?mcid=261N08...,2016,"[Crematory, Funeral Home, Mortuary]",5411 Okeechobee Blvd,West Palm Beach,FL,33417,None
10,Miller-Woodlawn Memorial Park,/canamembers/Cemetery/MillerWoodlawn-Memorial-...,NaN,(209) 466-8075,Mindy RochaBarella,/directory/directoryemailform.aspx?mcid=2S1S03...,2015,[],5505 Kitsap Way,Bremerton,WA,98312,None
1187,El Camino Memorial - Sorrento Valley,/canamembers/Funeral-Home/El-Camino-Memorial-S...,http://www.sorrentovalleychapel.com,(858) 453-2121,Douglas Trobaugh,/directory/directoryemailform.aspx?mcid=2F1U0T...,2015,"[Crematory, Funeral Home, Mortuary]",5600 Carroll Canyon Rd,San Diego,CA,92121,None


In [5]:
profile(df_locations_raw, "locations_raw")



=== locations_raw ===
shape: (8265, 9)


,id,title,latitude,longitude,description,phone,websiteUrl,address,directionsUrl
0,a93d2885-5b48-ef11-a317-7c1e5216a74f,1st Choice Mortuary,32.23935,-90.23272,,601-346-5738,,"3989 Terry Rd , Jackson, 39212-5266, US",
1,86e42e0e-f167-f011-bec2-000d3a180c28,302Funerals / Williams Funeral Home and Buria...,39.74206,-75.58487,,,https://www.302funerals.com/,"2100 Rodman Rd , Wilmington, 19805-4133, US",
2,27e9233e-93b8-db11-b90e-0050568763a2,A A Mariani & Son Funeral Home,41.85052,-71.42264,,401-861-5432,https://www.marianiandson.com/,"200 Hawkins St , Providence, 02904-2216, US",



Columns: ['id', 'title', 'latitude', 'longitude', 'description', 'phone', 'websiteUrl', 'address', 'directionsUrl']


,null_%
id,0.0
title,0.0
latitude,0.0
longitude,0.0
description,0.0
phone,0.0
websiteUrl,0.0
address,0.0
directionsUrl,0.0


,id,title,latitude,longitude,description,phone,websiteUrl,address,directionsUrl
3697,bef6d274-96b8-db11-b90e-0050568763a2,Ippolito-Stellato Funeral Home,40.806323,-74.120269,,201-438-4664,http://www.stellatofuneralhomes.com,"425 Ridge Rd , Lyndhurst, 07071-2406, US",
6414,339fc1f7-90b8-db11-b90e-0050568763a2,Roller-Farmers Union Funeral Home,35.83845,-90.73282,,870-932-3531,http://www.rollerfuneralhomes.com,"1900 W Washington Ave , Jonesboro, 72401-2577...",
8089,f8e79078-95b8-db11-b90e-0050568763a2,Wilson Funeral Home,37.56031,-90.2928,,573-783-6413,http://www.wilson-funeral-home.com,"100 N Mine La Motte St , Fredericktown, 6364...",
7093,9c986ac5-8fb8-db11-b90e-0050568763a2,Sterling White Funeral Home & Cemetery,29.82264,-95.04998,,281-426-3555,,"11011 Crosby-Lynchburg Rd , Highlands, 77562...",
3894,e11d0c6d-7f65-ef11-bfe2-002248230572,Johnson Funeral Home - Hampstead Chapel,34.38067,-77.69435,,910-319-8718,https://www.johnsonfhnc.com/,"16076 US Highway 17 , Hampstead, 28443-3398, US",
5688,75e48d66-95b8-db11-b90e-0050568763a2,Pagel Funeral Homes Ltd,38.91658,-88.66554,,618-238-4441,,"701 Broad St , Edgewood, 62426, US",
5413,1850c9d9-90b8-db11-b90e-0050568763a2,Nelson-Berna Funeral Home,36.33336,-94.16217,,479-631-1353,http://www.stockdalemoody.com,"100 S 28th St , Rogers, 72758-3901, US",
5623,87671771-97b8-db11-b90e-0050568763a2,Oman Funeral Home,36.71039,-76.26226,,757-547-5184,http://www.omanfh.com,"653 Cedar Rd , Chesapeake, 23322-8315, US",
4947,f35996a2-95b8-db11-b90e-0050568763a2,Melancon Funeral Home,30.30332,-92.04946,,337-896-6366,http://www.melanconfuneralhome.net,"4117 N University Ave , Carencro, 70520-4204...",
8018,1de30cb0-93b8-db11-b90e-0050568763a2,Wilkinson-Beane-Simoneau-Paquette FH & Crem. S...,43.52934,-71.47499,,603-524-4300,http://www.Wilkinsonbeane.com,"164 Pleasant St , Laconia, 03246-3030, US",


## Load legacy SQLite database

We will:
- list tables
- inspect columns
- load candidate tables into DataFrames

If your legacy DB has multiple relevant tables, we can join them here.


In [6]:
# --- Inspect legacy DB ---
con = sqlite3.connect(str(LEGACY_DB_PATH))
cur = con.cursor()

legacy_main = pd.read_sql_query("SELECT * FROM funeral_homes;", con)
profile(legacy_main, "legacy_db")


=== legacy_db ===
shape: (19130, 13)


,id,name,address,city,state,zip_code,country,price_range,rating,detail_url,image_url,listing_state,scraped_at
0,25891,Lawrence Brown-Service Funeral Home Inc,"2900 Citizens Pkwy, \n ...",,,36701,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/alabama/s...,https://d1ore3n0z2tnb3.cloudfront.net/assets/i...,alabama,2026-01-19 08:55:11
1,28822,"Randall Miller Funeral Service, Inc","608 St. Phillips St, P.O. Box 1471,\n ...",,,36703,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/alabama/s...,https://d1ore3n0z2tnb3.cloudfront.net/assets/i...,alabama,2026-01-19 08:55:11
2,27823,"Ashley's J.H. Williams & Sons, Inc.","1226 Minter Ave, \n ...",,,36703,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/alabama/s...,https://d1ore3n0z2tnb3.cloudfront.net/assets/i...,alabama,2026-01-19 08:55:11



Columns: ['id', 'name', 'address', 'city', 'state', 'zip_code', 'country', 'price_range', 'rating', 'detail_url', 'image_url', 'listing_state', 'scraped_at']


,null_%
zip_code,0.2
country,0.0
image_url,0.0
name,0.0
id,0.0
state,0.0
city,0.0
address,0.0
price_range,0.0
rating,0.0


,id,name,address,city,state,zip_code,country,price_range,rating,detail_url,image_url,listing_state,scraped_at
5331,18565,Hinton-Turner Funeral Home,"526 Pleasant Street, \n ...",,,40361,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/kentucky/...,https://prod-legacy.s3.amazonaws.com/summary_f...,kentucky,2026-01-19 09:29:08
12363,21564,"Moore, Snear & Ruggiero Funeral Home - Conshoh...","300 Fayette Street, \n ...",,,19428,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/pennsylva...,https://d1ore3n0z2tnb3.cloudfront.net/assets/i...,pennsylvania,2026-01-19 10:16:17
13103,21657,EARL L. FOSTER FUNERAL HOME - Chester,"1100 Kerlin Street, \n ...",,,19013,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/pennsylva...,https://prod-legacy.s3.amazonaws.com/summary_f...,pennsylvania,2026-01-19 10:21:46
1707,21582,Mealey Funeral Homes & Crematory,"2509 Limestone Rd, Limestone and Milltown Road...",,,19808,United States,$$$ - Moderate,4.5,https://www.legacy.com/funeral-homes/delaware/...,https://d1ore3n0z2tnb3.cloudfront.net/assets/i...,delaware,2026-01-19 09:05:30
4747,29616,Snyder & Hollenbaugh Funeral & Cremation Servi...,"603 2Nd St, \n ...",,,52738,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/iowa/colu...,https://prod-legacy.s3.amazonaws.com/summary_f...,iowa,2026-01-19 09:25:30
5992,19992,"Cremation Society of Maryland, Inc. & Metro Cr...","301 Frederick Rd, \n ...",,,21228,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/maryland/...,https://prod-legacy.s3.amazonaws.com/summary_f...,maryland,2026-01-19 09:33:17
2959,22404,Cumerford-Clary Funeral Home - Peoria,"2101 N. Knoxville Ave, \n ...",,,61603,United States,$$$ - Moderate,4.0,https://www.legacy.com/funeral-homes/illinois/...,https://d1ore3n0z2tnb3.cloudfront.net/assets/i...,illinois,2026-01-19 09:13:22
15909,32552,Heavner & Cutright Funeral Chapel,"940 Route 20 South Rd, \n ...",,,26201,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/west-virg...,https://d1ore3n0z2tnb3.cloudfront.net/assets/i...,west-virginia,2026-01-19 10:40:52
8368,24083,Brockhaus Funeral Home - Bloomfield,"220 East Main, \n ...",,,68718,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/nebraska/...,https://prod-legacy.s3.amazonaws.com/summary_f...,nebraska,2026-01-19 09:48:21
14009,28682,Holley-Gamble Funeral Home,"621 S Charles G Seivers Blvd, \n ...",,,37716,United States,$$$ - Moderate,0.0,https://www.legacy.com/funeral-homes/tennessee...,https://d1ore3n0z2tnb3.cloudfront.net/assets/i...,tennessee,2026-01-19 10:27:45


## Define a unified internal schema

We'll normalize everything to this schema:

- `internal_id` (assigned later)
- `name`
- `street`
- `city`
- `region` (state/province)
- `postal_code`
- `country`
- `phone`
- `website`
- `latitude`
- `longitude`
- `source` (`listings` / `locations` / `legacy`)
- `source_id` (best-effort identifier from the source)
- `raw` (raw JSON / raw row dict for traceability)

We also compute:
- `name_norm`
- `address_norm`
- `dedupe_key_block` (for blocking)


In [7]:
# --- Normalization helpers ---
def clean_text(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    x = str(x).strip()
    if x == "":
        return None
    return x

def norm_whitespace(x):
    x = clean_text(x)
    if not x:
        return None
    return re.sub(r"\s+", " ", x).strip()

def norm_phone(x):
    x = clean_text(x)
    if not x:
        return None
    digits = re.sub(r"\D+", "", x)
    if len(digits) < 7:
        return None
    # keep US-like digits, but don't force formatting
    return digits

def norm_url(x):
    x = clean_text(x)
    if not x:
        return None
    x = x.strip()
    if x and not re.match(r"^https?://", x, flags=re.I):
        # Accept bare domains like "www.example.com"
        if re.match(r"^[\w.-]+\.[a-z]{2,}(/.*)?$", x, flags=re.I):
            x = "http://" + x
    return x

_ABBREV = {
    "st": "street", "st.": "street",
    "rd": "road", "rd.": "road",
    "ave": "avenue", "ave.": "avenue",
    "blvd": "boulevard", "blvd.": "boulevard",
    "dr": "drive", "dr.": "drive",
    "hwy": "highway", "ln": "lane", "ln.": "lane",
    "ct": "court", "ct.": "court",
    "ste": "suite", "ste.": "suite", "#": "suite",
}

def norm_address_line(x):
    x = norm_whitespace(x)
    if not x:
        return None
    x = x.lower()
    # expand common abbreviations
    tokens = []
    for tok in re.split(r"([\s,./-]+)", x):
        if tok.strip() == "":
            tokens.append(tok)
            continue
        key = tok.strip()
        if key in _ABBREV:
            tokens.append(_ABBREV[key])
        else:
            tokens.append(tok)
    x = "".join(tokens)
    x = re.sub(r"[^a-z0-9\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x or None

def norm_name(x):
    x = norm_whitespace(x)
    if not x:
        return None
    x = x.lower()
    x = x.replace("&", " and ")
    x = re.sub(r"\b(dba|doing business as)\b", " ", x)
    x = re.sub(r"[^a-z0-9\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    # remove very common business suffixes
    x = re.sub(r"\b(inc|llc|ltd|co|company|corp|corporation)\b\.?$", "", x).strip()
    return x or None

def safe_float(x):
    x = clean_text(x)
    if not x:
        return None
    try:
        return float(x)
    except:
        return None

def parse_location_address(addr):
    """Parse the locations JSON 'address' which looks like:
    '3989 Terry Rd , Jackson,  39212-5266, US'
    Not perfect, but good enough for normalization + dedupe.
    """
    addr = norm_whitespace(addr)
    if not addr:
        return (None, None, None, None, None)
    parts = [p.strip() for p in addr.split(",")]
    # Heuristic: street, city, (maybe region), postal, country
    street = parts[0] if len(parts) >= 1 else None
    city = parts[1] if len(parts) >= 2 else None
    # Some have: street, city, postal, country (no state)
    region = None
    postal = None
    country = None
    tail = [p for p in parts[2:] if p]
    if len(tail) == 0:
        pass
    elif len(tail) == 1:
        # could be postal or country
        if re.search(r"\d", tail[0]):
            postal = tail[0]
        else:
            country = tail[0]
    elif len(tail) == 2:
        # postal, country OR region, postal
        if re.search(r"\d", tail[0]) and not re.search(r"\d", tail[1]):
            postal, country = tail
        else:
            region, postal = tail
    else:
        # region, postal, country (or extra noise)
        region = tail[0]
        postal = tail[1]
        country = tail[-1]
    # normalize bits
    street = norm_whitespace(street)
    city = norm_whitespace(city)
    region = norm_whitespace(region)
    postal = norm_whitespace(postal)
    country = norm_whitespace(country)
    return street, city, region, postal, country


In [8]:
# --- Normalize: listings JSON ---
def normalize_listings(records):
    rows = []
    for r in records:
        addr = r.get("address") or {}
        row = {
            "name": clean_text(r.get("name")),
            "street": clean_text(addr.get("street")),
            "city": clean_text(addr.get("locality")),
            "region": clean_text(addr.get("region")),
            "postal_code": clean_text(addr.get("postalCode")),
            "country": clean_text(addr.get("country")),
            "phone": norm_phone(r.get("phone")),
            "website": norm_url(r.get("website")),
            "latitude": None,
            "longitude": None,
            "source": "listings",
            "source_id": clean_text(r.get("learnMoreLink")) or clean_text(r.get("contactLink")),
            "raw": r,
        }
        rows.append(row)
    df = pd.DataFrame(rows)
    return df

# --- Normalize: locations JSON ---
def normalize_locations(records):
    rows = []
    for r in records:
        street, city, region, postal, country = parse_location_address(r.get("address"))
        row = {
            "name": clean_text(r.get("title")),
            "street": street,
            "city": city,
            "region": region,
            "postal_code": postal,
            "country": country,
            "phone": norm_phone(r.get("phone")),
            "website": norm_url(r.get("websiteUrl")),
            "latitude": safe_float(r.get("latitude")),
            "longitude": safe_float(r.get("longitude")),
            "source": "locations",
            "source_id": clean_text(r.get("id")),
            "raw": r,
        }
        rows.append(row)
    df = pd.DataFrame(rows)
    return df

df_listings = normalize_listings(listings_raw)
df_locations = normalize_locations(locations_raw)

print(df_listings.shape, df_locations.shape)
display(df_listings.head(3))
display(df_locations.head(3))


(3019, 13) (8265, 13)


,name,street,city,region,postal_code,country,phone,website,latitude,longitude,source,source_id,raw
0,El Camino Memorial Park,5600 Carroll Canyon Rd,San Diego,CA,92121,None,7186330400,NaN,None,None,listings,/canamembers/Cemetery/El-Camino-Memorial-Park-...,"{'name': 'El Camino Memorial Park', 'learnMore..."
1,Cementerio Parque De Luz,"Carr 1, KM 29.2",Caguas,PR,00726 1569,None,7877453200,http://www.cementerioparquedeluz.com,None,None,listings,/canamembers/Cemetery/Cementerio-Parque-De-Luz...,"{'name': 'Cementerio Parque De Luz', 'learnMor..."
2,Coral Ridge Funeral Home & Cemetery,1630 SW Pine Island Rd,Cape Coral,FL,33991,None,4236982541,NaN,None,None,listings,/canamembers/Cemetery/Coral-Ridge-Funeral-Home...,{'name': 'Coral Ridge Funeral Home & Cemetery'...


,name,street,city,region,postal_code,country,phone,website,latitude,longitude,source,source_id,raw
0,1st Choice Mortuary,3989 Terry Rd,Jackson,NaN,39212-5266,US,6013465738,NaN,32.23935,-90.23272,locations,a93d2885-5b48-ef11-a317-7c1e5216a74f,"{'id': 'a93d2885-5b48-ef11-a317-7c1e5216a74f',..."
1,302Funerals / Williams Funeral Home and Buria...,2100 Rodman Rd,Wilmington,NaN,19805-4133,US,NaN,https://www.302funerals.com/,39.74206,-75.58487,locations,86e42e0e-f167-f011-bec2-000d3a180c28,"{'id': '86e42e0e-f167-f011-bec2-000d3a180c28',..."
2,A A Mariani & Son Funeral Home,200 Hawkins St,Providence,NaN,02904-2216,US,4018615432,https://www.marianiandson.com/,41.85052,-71.42264,locations,27e9233e-93b8-db11-b90e-0050568763a2,"{'id': '27e9233e-93b8-db11-b90e-0050568763a2',..."


In [9]:
# --- Normalize: legacy DB ---
# We try to map columns automatically. Adjust this mapping if needed after inspecting 'legacy_main'.
def pick_col(df, *candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        for c_lower, c_orig in cols.items():
            if c_lower == cand.lower():
                return c_orig
        # partial match
        for c_lower, c_orig in cols.items():
            if cand.lower() in c_lower:
                return c_orig
    return None

def normalize_legacy(df):
    if df is None or df.empty:
        return pd.DataFrame(columns=["name","street","city","region","postal_code","country","phone","website","latitude","longitude","source","source_id","raw"])

    name_c = pick_col(df, "name", "title", "funeral_home_name")
    street_c = pick_col(df, "street", "address1", "address", "street_address", "addr")
    city_c = pick_col(df, "city", "locality", "town")
    region_c = pick_col(df, "state", "region", "province")
    postal_c = pick_col(df, "zip", "zipcode", "postal", "postal_code")
    country_c = pick_col(df, "country")
    phone_c = pick_col(df, "phone", "phone_number", "tel")
    website_c = pick_col(df, "website", "url", "web")
    lat_c = pick_col(df, "latitude", "lat")
    lon_c = pick_col(df, "longitude", "lon", "lng")

    id_c = pick_col(df, "id", "legacy_id", "uuid")

    rows = []
    for _, r in df.iterrows():
        raw = r.to_dict()
        row = {
            "name": clean_text(raw.get(name_c)) if name_c else None,
            "street": clean_text(raw.get(street_c)) if street_c else None,
            "city": clean_text(raw.get(city_c)) if city_c else None,
            "region": clean_text(raw.get(region_c)) if region_c else None,
            "postal_code": clean_text(raw.get(postal_c)) if postal_c else None,
            "country": clean_text(raw.get(country_c)) if country_c else None,
            "phone": norm_phone(raw.get(phone_c)) if phone_c else None,
            "website": norm_url(raw.get(website_c)) if website_c else None,
            "latitude": safe_float(raw.get(lat_c)) if lat_c else None,
            "longitude": safe_float(raw.get(lon_c)) if lon_c else None,
            "source": "legacy",
            "source_id": clean_text(raw.get(id_c)) if id_c else None,
            "raw": raw,
        }
        rows.append(row)

    return pd.DataFrame(rows)

df_legacy = normalize_legacy(legacy_main)
print("legacy normalized shape:", df_legacy.shape)
df_legacy.head(3)


legacy normalized shape: (19130, 13)


,name,street,city,region,postal_code,country,phone,website,latitude,longitude,source,source_id,raw
0,Lawrence Brown-Service Funeral Home Inc,"2900 Citizens Pkwy, \n ...",None,NaN,36701,United States,None,https://www.legacy.com/funeral-homes/alabama/s...,None,None,legacy,25891,"{'id': '25891', 'name': 'Lawrence Brown-Servic..."
1,"Randall Miller Funeral Service, Inc","608 St. Phillips St, P.O. Box 1471,\n ...",None,NaN,36703,United States,None,https://www.legacy.com/funeral-homes/alabama/s...,None,None,legacy,28822,"{'id': '28822', 'name': 'Randall Miller Funera..."
2,"Ashley's J.H. Williams & Sons, Inc.","1226 Minter Ave, \n ...",None,NaN,36703,United States,None,https://www.legacy.com/funeral-homes/alabama/s...,None,None,legacy,27823,"{'id': '27823', 'name': 'Ashley's J.H. William..."


In [10]:
# --- Combine all normalized sources ---
df_all = pd.concat([df_listings, df_locations, df_legacy], ignore_index=True)

# Add normalized fields for matching
df_all["name_norm"] = df_all["name"].map(norm_name)
df_all["street_norm"] = df_all["street"].map(norm_address_line)
df_all["city_norm"] = df_all["city"].map(lambda x: norm_whitespace(x).lower() if norm_whitespace(x) else None)
df_all["region_norm"] = df_all["region"].map(lambda x: norm_whitespace(x).upper() if norm_whitespace(x) else None)
df_all["postal_norm"] = df_all["postal_code"].map(lambda x: re.sub(r"\s+", "", str(x)).upper() if clean_text(x) else None)

df_all["address_norm"] = (
    df_all["street_norm"].fillna("")
    + "|" + df_all["city_norm"].fillna("")
    + "|" + df_all["region_norm"].fillna("")
    + "|" + df_all["postal_norm"].fillna("")
)

# Simple blocking key: (region, first3 of postal) or (region, city) if postal missing
def _s(x) -> str:
    # normalize NaN/None to empty string
    if x is None or (isinstance(x, float) and pd.isna(x)) or pd.isna(x):
        return ""
    return str(x)

def make_block(r):
    reg = _s(r.get("region_norm", "")).strip()
    postal = _s(r.get("postal_norm", "")).strip()
    city = _s(r.get("city_norm", "")).strip()

    if postal:
        return f"{reg}|{postal[:3]}"
    if city:
        return f"{reg}|{city[:6]}"
    return reg or "UNK"

df_all["dedupe_key_block"] = df_all.apply(make_block, axis=1)

print("Combined:", df_all.shape)
df_all.head(3)


Combined: (30414, 20)


,name,street,city,region,postal_code,country,phone,website,latitude,longitude,source,source_id,raw,name_norm,street_norm,city_norm,region_norm,postal_norm,address_norm,dedupe_key_block
0,El Camino Memorial Park,5600 Carroll Canyon Rd,San Diego,CA,92121,None,7186330400,NaN,None,None,listings,/canamembers/Cemetery/El-Camino-Memorial-Park-...,"{'name': 'El Camino Memorial Park', 'learnMore...",el camino memorial park,5600 carroll canyon road,san diego,CA,92121,5600 carroll canyon road|san diego|CA|92121,CA|921
1,Cementerio Parque De Luz,"Carr 1, KM 29.2",Caguas,PR,00726 1569,None,7877453200,http://www.cementerioparquedeluz.com,None,None,listings,/canamembers/Cemetery/Cementerio-Parque-De-Luz...,"{'name': 'Cementerio Parque De Luz', 'learnMor...",cementerio parque de luz,carr 1 km 29 2,caguas,PR,007261569,carr 1 km 29 2|caguas|PR|007261569,PR|007
2,Coral Ridge Funeral Home & Cemetery,1630 SW Pine Island Rd,Cape Coral,FL,33991,None,4236982541,NaN,None,None,listings,/canamembers/Cemetery/Coral-Ridge-Funeral-Home...,{'name': 'Coral Ridge Funeral Home & Cemetery'...,coral ridge funeral home and cemetery,1630 sw pine island road,cape coral,FL,33991,1630 sw pine island road|cape coral|FL|33991,FL|339


## Deduplication strategy

Because names/addresses may differ slightly, we use a two-stage approach:

1. **Blocking**: only compare records within the same block (e.g., same state + first 3 digits of ZIP; or state + city prefix).
2. **Fuzzy scoring**: compute similarity scores for `name_norm` and `street_norm`.
3. **Merge rule**: if scores exceed thresholds (or phone/website match strongly), treat as same entity.


In [11]:
# --- Deduplication ---
from rapidfuzz.fuzz import token_set_ratio, partial_ratio


def fuzz_ratio(a, b):
    if not a or not b:
        return 0
    else:
        # token_set_ratio is good for re-orderings and extra tokens
        return token_set_ratio(a, b)

def fuzz_partial(a, b):
    if not a or not b:
        return 0
    else:
        return partial_ratio(a, b)

def strong_exact_match(a, b):
    a = clean_text(a); b = clean_text(b)
    if not a or not b:
        return False
    return a.strip().lower() == b.strip().lower()

# Candidate pair generation within blocks
df_all = df_all.reset_index(drop=True)
df_all["row_id"] = df_all.index.astype(int)

# thresholds (tune as needed)
NAME_OK = 90
ADDR_OK = 90
NAME_PARTIAL_OK = 95

def is_match(r1, r2):
    # strong exact signals
    if strong_exact_match(r1.get("phone"), r2.get("phone")) and r1.get("phone") is not None:
        return True
    if strong_exact_match(r1.get("website"), r2.get("website")) and r1.get("website") is not None:
        return True

    name_score = fuzz_ratio(r1["name_norm"], r2["name_norm"])
    addr_score = fuzz_ratio(r1["street_norm"], r2["street_norm"])
    name_partial = fuzz_partial(r1["name_norm"], r2["name_norm"])

    # If both reasonably close
    if name_score >= NAME_OK and addr_score >= ADDR_OK:
        return True
    # Sometimes street differs (PO Box vs street); allow strong name + same city/region
    if name_partial >= NAME_PARTIAL_OK:
        same_city = (r1["city_norm"] and r2["city_norm"] and r1["city_norm"] == r2["city_norm"])
        same_reg = (r1["region_norm"] and r2["region_norm"] and r1["region_norm"] == r2["region_norm"])
        if same_city and same_reg:
            return True
    return False

# Union-Find (disjoint set) for clustering duplicates
parent = {i: i for i in df_all["row_id"].tolist()}

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

# Compare within each block
blocks = df_all.groupby("dedupe_key_block")["row_id"].apply(list).to_dict()
total_pairs = 0
matched_pairs = 0

for block, ids in blocks.items():
    if len(ids) < 2:
        continue
    # naive O(n^2) within block; acceptable for take-home sized datasets
    # if blocks are large, you can add secondary blocking on name initial, etc.
    for i in range(len(ids)):
        r1 = df_all.loc[ids[i]]
        for j in range(i+1, len(ids)):
            r2 = df_all.loc[ids[j]]
            total_pairs += 1
            if is_match(r1, r2):
                matched_pairs += 1
                union(int(r1["row_id"]), int(r2["row_id"]))

print("Blocks:", len(blocks))
print("Compared pairs:", total_pairs)
print("Matched pairs:", matched_pairs)

df_all["cluster_id"] = df_all["row_id"].map(find)
df_all[["row_id","source","name","street","city","region","postal_code","cluster_id"]].head(10)


Blocks: 1504
Compared pairs: 672295
Matched pairs: 7850


,row_id,source,name,street,city,region,postal_code,cluster_id
0,0,listings,El Camino Memorial Park,5600 Carroll Canyon Rd,San Diego,CA,92121,338
1,1,listings,Cementerio Parque De Luz,"Carr 1, KM 29.2",Caguas,PR,00726 1569,1
2,2,listings,Coral Ridge Funeral Home & Cemetery,1630 SW Pine Island Rd,Cape Coral,FL,33991,2
3,3,listings,Rosewood Memorial Park,631 North Witchduck Road,Virginia Beach,VA,23462,3
4,4,listings,"Moriconi Funeral Home, Inc. dba Cremation Solu...",1303 Bigler Ave,Northern Cambria,PA,15714,4
5,5,listings,Advantage Funeral & Cremation Services,4408 West Central,Wichita,KS,67212,5
6,6,listings,Ridgelawn Memorial Park,5547 West Pea Ridge Road,Huntington,WV,25705,6
7,7,listings,Green Acres Memorial Park & Mortuary,11715 Cedar Ave,Bloomington,CA,92316,7
8,8,listings,Desert Lawn Memorial Park,1415 S 1st Ave,Yuma,AZ,85364 4751,8
9,9,listings,Omega Funeral & Cremation Service,223 SE 122nd Ave,Portland,OR,97233 1005,9


In [12]:
# Cluster stats
cluster_sizes = df_all.groupby("cluster_id").size().sort_values(ascending=False)
print("Total rows:", len(df_all))
print("Unique clusters:", cluster_sizes.shape[0])
display(cluster_sizes.head(20).to_frame("cluster_size"))


Total rows: 30414
Unique clusters: 23230


,cluster_size
cluster_id,
5543,19
10828,16
7271,14
3199,11
9476,11
4123,10
8877,10
3056,10
9309,10


## Create canonical (merged) record per cluster

We choose the "best" value per field by:
- preferring non-null
- preferring longer strings (more informative)
- for lat/lon: prefer values that exist
- keep traceability: list of source rows


In [13]:
def pick_best(values):
    # values is a list/Series
    vals = [clean_text(v) for v in values if clean_text(v)]
    if not vals:
        return None
    # prefer the longest (often the most complete)
    return sorted(vals, key=lambda x: (len(x), x), reverse=True)[0]

def pick_best_float(values):
    vals = [safe_float(v) for v in values if safe_float(v) is not None]
    if not vals:
        return None
    # median is robust to outliers
    return float(np.median(vals))

def merge_cluster(g):
    return pd.Series({
        "name": pick_best(g["name"]),
        "street": pick_best(g["street"]),
        "city": pick_best(g["city"]),
        "region": pick_best(g["region"]),
        "postal_code": pick_best(g["postal_code"]),
        "country": pick_best(g["country"]),
        "phone": pick_best(g["phone"]),
        "website": pick_best(g["website"]),
        "latitude": pick_best_float(g["latitude"]),
        "longitude": pick_best_float(g["longitude"]),
        # provenance
        "sources": sorted(set(g["source"].dropna().tolist())),
        "source_ids": [x for x in g["source_id"].dropna().tolist()],
        "raw_rows": g["raw"].tolist(),
        "cluster_size": int(len(g)),
    })

df_canonical = df_all.groupby("cluster_id", as_index=False).apply(merge_cluster).reset_index(drop=True)

# assign internal_id
df_canonical.insert(0, "internal_id", np.arange(1, len(df_canonical)+1))
print("Canonical rows:", df_canonical.shape)
df_canonical.head(5)


Canonical rows: (23230, 16)


,internal_id,cluster_id,name,street,city,region,postal_code,country,phone,website,latitude,longitude,sources,source_ids,raw_rows,cluster_size
0,1,1,Cementerio Parque De Luz,"Carr 1, KM 29.2",Caguas,PR,00726 1569,NaN,7877453200,http://www.cementerioparquedeluz.com,NaN,NaN,[listings],[/canamembers/Cemetery/Cementerio-Parque-De-Lu...,"[{'name': 'Cementerio Parque De Luz', 'learnMo...",1
1,2,2,Coral Ridge Funeral Home & Cemetery,1630 SW Pine Island Road,Cape Coral,FL,33991 2150,NaN,4236982541,http://www.coralridgefuneralhome.com,NaN,NaN,[listings],[/canamembers/Cemetery/Coral-Ridge-Funeral-Hom...,[{'name': 'Coral Ridge Funeral Home & Cemetery...,2
2,3,3,Rosewood Memorial Park,631 North Witchduck Road,Virginia Beach,VA,23462,NaN,4349770005,NaN,NaN,NaN,[listings],[/canamembers/Cemetery/Rosewood-Memorial-Park-...,"[{'name': 'Rosewood Memorial Park', 'learnMore...",1
3,4,4,"Moriconi Funeral Home, Inc. dba Cremation Solu...",1303 Bigler Ave,Northern Cambria,PA,15714,NaN,8149488050,http://www.moriconifuneralhome.com,NaN,NaN,[listings],[/canamembers/Funeral-Home/Moriconi-Funeral-Ho...,"[{'name': 'Moriconi Funeral Home, Inc. dba Cre...",1
4,5,5,Advantage Funeral & Cremation Services,4408 West Central,Wichita,KS,67212,NaN,3067578645,NaN,NaN,NaN,[listings],[/canamembers/Funeral-Home/Advantage-Funeral-C...,[{'name': 'Advantage Funeral & Cremation Servi...,1


## Quality checks

We’ll inspect:
- missing rates in the canonical data
- obvious invalid lat/lon
- distribution by region


In [14]:
null_rate = (df_canonical.isna().mean().sort_values(ascending=False) * 100).round(1)
display(null_rate.to_frame("null_%"))

# lat/lon sanity
bad_geo = df_canonical[
    (df_canonical["latitude"].notna() & ((df_canonical["latitude"] < -90) | (df_canonical["latitude"] > 90))) |
    (df_canonical["longitude"].notna() & ((df_canonical["longitude"] < -180) | (df_canonical["longitude"] > 180)))
]
print("Bad geo rows:", len(bad_geo))
display(bad_geo.head(10))

display(df_canonical["region"].value_counts(dropna=False).head(20).to_frame("count"))


,null_%
region,88.5
longitude,69.2
latitude,69.2
phone,58.5
city,57.7
country,11.5
website,6.0
postal_code,0.1
internal_id,0.0
cluster_id,0.0


Bad geo rows: 0


,internal_id,cluster_id,name,street,city,region,postal_code,country,phone,website,latitude,longitude,sources,source_ids,raw_rows,cluster_size


,count
region,
NaN,20551
FL,270
CA,255
TX,245
IL,125
NY,125
OH,99
PA,97
VA,91


## Store unified data into a SQLite database

We will create a new SQLite DB at `data/funeral_homes_unified.db` with:
- `funeral_homes` (canonical records)
- `funeral_home_sources` (one row per original record, linked to canonical internal_id)



In [15]:
from sqlalchemy import create_engine

OUT_SQLITE_PATH = Path("data/funeral_homes_unified.db")
if OUT_SQLITE_PATH.exists():
    OUT_SQLITE_PATH.unlink()

engine = create_engine(f"sqlite:///{OUT_SQLITE_PATH}")

# Canonical table (flatten types)
canonical_out = df_canonical.drop(columns=["raw_rows"]).copy()
canonical_out["sources"] = canonical_out["sources"].apply(lambda x: json.dumps(x, ensure_ascii=False))
canonical_out["source_ids"] = canonical_out["source_ids"].apply(lambda x: json.dumps(x, ensure_ascii=False))

canonical_out.to_sql("funeral_homes", engine, index=False)

# Sources table
# Link each original row to internal_id via cluster_id mapping
cluster_to_internal = dict(zip(df_canonical["cluster_id"], df_canonical["internal_id"]))
sources_out = df_all.copy()
sources_out["internal_id"] = sources_out["cluster_id"].map(cluster_to_internal)

# serialize raw dict
sources_out["raw"] = sources_out["raw"].apply(lambda x: json.dumps(x, ensure_ascii=False))
sources_out = sources_out[[
    "internal_id","source","source_id",
    "name","street","city","region","postal_code","country","phone","website","latitude","longitude",
    "name_norm","street_norm","city_norm","region_norm","postal_norm","address_norm","dedupe_key_block","cluster_id",
    "raw"
]]

sources_out.to_sql("funeral_home_sources", engine, index=False)

print("Wrote SQLite:", OUT_SQLITE_PATH)
print("funeral_homes rows:", canonical_out.shape[0])
print("funeral_home_sources rows:", sources_out.shape[0])

# Quick SQL sanity checks
with engine.connect() as c:
    n1 = c.exec_driver_sql("SELECT COUNT(*) FROM funeral_homes;").fetchone()[0]
    n2 = c.exec_driver_sql("SELECT COUNT(*) FROM funeral_home_sources;").fetchone()[0]
print("Counts from DB:", n1, n2)


Wrote SQLite: data/funeral_homes_unified.db
funeral_homes rows: 23230
funeral_home_sources rows: 30414
Counts from DB: 23230 30414


## Next improvements (if you have time)

- Improve address parsing (USPS-style normalization, unit/suite extraction).
- Use `recordlinkage` or `splink` for scalable, probabilistic linkage.
- Add geospatial proximity (e.g., Haversine distance) as a matching feature when lat/lon exist.
- Introduce deterministic unique keys (e.g., website domain + street+zip) to reduce fuzzy matching.
- Add constraints + indexes in the output DB (e.g., index on `region`, `postal_code`, `name`).
